In [1]:
import abc

In [2]:
class Dialog(abc.ABC):
    def listen(self, text, response=True, **kwargs):
        # Parse the input
        sents = self.parse(text)
        
        # Interpret the input
        sents, confidence, kwargs = self.interpret(sents, **kwargs)
        
        # Determine the response
        if response:
            reply = self.respond(sents, confidence, **kwargs)
        else:
            reply = None
            
        # Return initiative
        return reply, confidence

In [3]:
@abc.abstractmethod
def parse(self, text):
    return []

In [4]:
@abc.abstractmethod
def interpret(self, sents, **kwargs):
    return sents, 0.0, kwargs

In [5]:
@abc.abstractmethod
def interpret(self, sents, **kwargs):
    return None

On `Conversation.listen` we will pass the incoming text to each of the internal `Dialog.listen` methods, which will in turn call the internal `Dialog` object's `parse, interpret`, and `response` methods. The result is a list of `(response, confidence)` tuples, and the `SimpleConversation` will return the response with the highest confidence, by using the `itemgetter` operator to retrieve the max by the second element of the tuple.

Because `SimpleConversation` is a `Dialog`, it *must* implement `parse, interpret` and `respond`.

In [6]:
from collections.abc import Sequence
from operator import itemgetter

class SimpleConversation(Dialog, Sequence):
    def __init__(self, dialogs):
        self._dialogs = dialogs
        
    def __getitem__(self, idx):
        return self.dialogs(idx)
    
    def __len__(self):
        return len(self._dialogs)
    
    def listen(self, text, response=True, **kwargs):
        responses = [
            dialog.listen(text, response, **kwargs)
            for dialog in self._dialogs
        ]
        
        # Responses is a list of (response, confidence) pairs
        return max(responses, key=itemgetter[1])
    
    def parse(self, text):
        return [dialog.parse(text) for dialog in self._dialogs]
    
    def interpret(self, sents, **kwargs):
        return [dialog.interpret(sents, **kwargs) for dialog in self._dialogs]
    
    def respond(self, sents, confidence, **kwargs):
        return [
            dialog.respond(sents, confidence, **kwargs)
            for dialog in self._dialogs
        ]

Now we will implement a rules-based greeting feature which uses regular expressions to match utterances. This version will maintain state primarily to acknowledge participants entering and leaving the dialog, and respond to them with appropriate salutations and questions.

In [14]:
import re

class Greeting(Dialog):
    PATTERNS = {
        'greeting': r'hello|hi|hey|good morning|good evening',
        'introduction': r'my name is ([a-z\-\s]+)',
        'goodbye': r'goodbye|bye|ttyl',
        'rollcall': r'roll call|who\'s here?'
    }
    
    def __init__(self, participants=None):
        self.participants = {}
        
        if participants is not None:
            for participant in participants:
                self.participants[participant] = None
                
        # Compile regular expressions
        self._patterns = {
            key: re.compile(pattern, re.I)
            for key, pattern in self.PATTERNS.items()
        }
        
    def parse(self, text):
        matches = {}
        for key, pattern in self._patterns.items():
            match = pattern.match(text)
            if match is not None:
                matches[key] = match
        return matches
    
    def interpret(self, sents, **kwargs):
        # Can't do anything with no matches
        if len(sents) == 0:
            return sents, 0.0, kwargs
        
        # Get username from the participants
        user = kwargs.get('user', None)
        
        # Determine if an introductions has been made
        if 'introduction' in sents:
            # Get the name from the utterance
            name = sents['introduction'].groups()[0]
            user = user or name.lower()
            
            # Determine if name has changed
            if user not in self.participants or self.participants[user] != name:
                kwargs['name_changed'] = True
                
            # Update the participants
            self.participants[user] = name
            kwargs['user'] = user
            
        # Determine if a greeting has been made
        if 'greeting' in sents:
            # If we don't have a name for the user
            if not self.participants.get(user, None):
                kwargs['request_introduction'] = True
                
        # Determine if goodbye has been made
        if 'goodbye' in sents and user is not None:
            # Remove participant
            self.participants.pop(user)
            kwargs.pop('user', None)
            
        # If we've seen anything we're looking for, we're pretty confident.
        return sents, 1.0, kwargs
    
    def respond(self, sents, confidence, **kwargs):
        if confidence == 0:
            return None
        
        name = self.participants.get(kwargs.get('user', None), None)
        name_changed = kwargs.get('name_changed', False)
        request_introduction = kwargs.get('request_introduction', False)
        
        if 'greeting' in sents or 'introduction' in sents:
            if request_introduction:
                return 'Hello, what is your name?'
            else:
                return "Hello, {}!".format(name)
            
        if 'goodbye' in sents:
            return 'Talk to you later!'
        
        if 'rollcall' in sents:
            people = list(self.participants.values())
            
            if len(people) > 1:
                roster = ", ".join(people[:-1])
                roster += " and {}.".format(people[-1])
                return "Currently in the conversation are " + roster
            
            elif len(people) == 1:
                return "It's just you and me right now, {}".format(name)
            else:
                return"So lonely in here by myself... Wait, who is that?"
            
        raise Exception("Expected response to be returned, but could not find rule.")        

In [15]:
dialog = Greeting()

print(dialog.listen("hey", user="David")[0])

Hello, what is your name?


In [16]:
print(dialog.listen("My name is David", user="David")[0])

Hello, David!


In [17]:
print(dialog.listen("Roll call!", user="David")[0])

It's just you and me right now, David


In [21]:
print(dialog.listen("Very well, goodbye!", user="David")[0])

None
